In [1]:
from query_api import query_api
from schedule_parser import TimetableProcessor


In [4]:
processor = TimetableProcessor(database_type="sqlite")  # or "csv"
current_events = processor.format_events_as_llm_output(include_header=False, limit=20)
print(current_events)

事项: 写代码
日期: 2025-03-06
时间段: 19:30-20:30
类型: 任务事项
截止日期：2025-03-06
重要程度：3

事项: 与周姐开会
日期: 2025-03-07
时间段: 09:00-10:00
类型: 固定日程
截止日期：2025-03-07
重要程度：5

事项: 准备小组讨论
日期: 2025-03-07
时间段: 10:30-11:30
类型: 任务事项
截止日期：2025-03-07
重要程度：4

事项: 参加小组讨论
日期: 2025-03-07
时间段: 15:30-17:30
类型: 固定日程
截止日期：2025-03-07
重要程度：5

事项: 洗漱
日期: 2025-03-08
时间段: 06:00-06:30
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 早餐
日期: 2025-03-08
时间段: 07:00-07:30
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 午餐
日期: 2025-03-08
时间段: 12:00-13:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 晚餐
日期: 2025-03-08
时间段: 18:00-19:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 睡觉
日期: 2025-03-08
时间段: 22:00-06:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 洗漱
日期: 2025-03-09
时间段: 06:00-06:30
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 早餐
日期: 2025-03-09
时间段: 07:00-07:30
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 午餐
日期: 2025-03-09
时间段: 12:00-13:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 晚餐
日期: 2025-03-09
时间段: 18:00-19:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 睡觉
日期: 2025-03-09
时间段: 22:00-06:00
类型: 任务事项
截止日期

In [4]:
import time

processor = TimetableProcessor(database_type="sqlite")  # or "csv"
current_events = processor.format_events_as_llm_output(include_header=False)
user_prompt = input("你有什么想放进计划表的东西吗？")
print(f"你的输入：{user_prompt}")
start_time = time.time()
response = query_api(user_prompt, current_events, model="deepseek-chat")
#response = query_api(user_prompt, current_events, model="gpt-4o")
end_time = time.time()
print(f"模型推理时间：{end_time - start_time}秒")
print("\n模型回复：")
print(response)

你的输入：帮我安排每日必做的规划，包括睡觉，洗漱，吃饭，要求时间安排合理，给出一整天的安排，选择目前没有安排的一天给出计划
模型推理时间：24.57064723968506秒

模型回复：
新任务：每日必做规划
每日必做规划 → 拆分为：
- 睡觉（预计需8小时，时段：2025-03-08 22:00-06:00）
- 洗漱（预计需30分钟，时段：2025-03-08 06:00-06:30）
- 早餐（预计需30分钟，时段：2025-03-08 07:00-07:30）
- 午餐（预计需1小时，时段：2025-03-08 12:00-13:00）
- 晚餐（预计需1小时，时段：2025-03-08 18:00-19:00）

日程建议：
事项: 睡觉
日期: 2025-03-08
时间段: 22:00-06:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5
变动：新增

事项: 洗漱
日期: 2025-03-08
时间段: 06:00-06:30
类型: 任务事项
截止日期：2025-03-08
重要程度：5
变动：新增

事项: 早餐
日期: 2025-03-08
时间段: 07:00-07:30
类型: 任务事项
截止日期：2025-03-08
重要程度：5
变动：新增

事项: 午餐
日期: 2025-03-08
时间段: 12:00-13:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5
变动：新增

事项: 晚餐
日期: 2025-03-08
时间段: 18:00-19:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5
变动：新增


In [5]:
#processor = TimetableProcessor()
#result = processor.process_recurring_events(response, recurrence_rule='daily')

# 将事件ID为5的事件设置为每周重复，直到指定日期
#result = processor.apply_recurrence_to_event(5, recurrence_rule='weekly', end_date='2024-12-31')

#recurring_events = processor.get_recurring_events()

#success = processor.remove_recurrence(5)

Extracted 5 events from LLM output
Event 1: 睡觉 - 2025-03-08 - 新增
Event 2: 洗漱 - 2025-03-08 - 新增
Event 3: 早餐 - 2025-03-08 - 新增
Event 4: 午餐 - 2025-03-08 - 新增
Event 5: 晚餐 - 2025-03-08 - 新增


In [6]:
# 获取修改前的所有事件
old_events = processor.get_all_events()

# 进行一些修改操作...
summary = processor.process_events(response)
print(summary)

# 获取修改后的所有事件
new_events = processor.get_all_events()

# 显示变更
changes = processor.format_events_with_changes(old_events, new_events, include_header=True)
print(changes)

Extracted 5 events from LLM output
Event 1: 睡觉 - 2025-03-08 - 新增
Event 2: 洗漱 - 2025-03-08 - 新增
Event 3: 早餐 - 2025-03-08 - 新增
Event 4: 午餐 - 2025-03-08 - 新增
Event 5: 晚餐 - 2025-03-08 - 新增
{'added': 0, 'modified': 0, 'deleted': 0, 'unchanged': 0, 'skipped': 5, 'errors': [], 'warnings': ["Skipped duplicate event: '睡觉' already exists with identical details", "Skipped duplicate event: '洗漱' already exists with identical details", "Skipped duplicate event: '早餐' already exists with identical details", "Skipped duplicate event: '午餐' already exists with identical details", "Skipped duplicate event: '晚餐' already exists with identical details"]}
日程变更明细：

----------------------------------------

[ ] 事项: 写代码
    日期: 2025-03-06
    时间段: 19:30-20:30
    类型: 任务事项
    截止日期：2025-03-06
    重要程度：3

[ ] 事项: 与周姐开会
    日期: 2025-03-07
    时间段: 09:00-10:00
    类型: 固定日程
    截止日期：2025-03-07
    重要程度：5

[ ] 事项: 准备小组讨论
    日期: 2025-03-07
    时间段: 10:30-11:30
    类型: 任务事项
    截止日期：2025-03-07
    重要程度：4

[ ] 事项: 参加小组讨

In [6]:
# This gets all the events in the database
formatted_output = processor.format_events_as_llm_output(limit=20)
print(formatted_output)

# This gets all the events for a specific date
today_events = processor.get_events_for_date("2025-03-03")
today_formatted = processor.format_events_as_llm_output(today_events)
#print(today_formatted)

事项: 写代码
日期: 2025-03-06
时间段: 19:30-20:30
类型: 任务事项
截止日期：2025-03-06
重要程度：3

事项: 与周姐开会
日期: 2025-03-07
时间段: 09:00-10:00
类型: 固定日程
截止日期：2025-03-07
重要程度：5

事项: 准备小组讨论
日期: 2025-03-07
时间段: 10:30-11:30
类型: 任务事项
截止日期：2025-03-07
重要程度：4

事项: 参加小组讨论
日期: 2025-03-07
时间段: 15:30-17:30
类型: 固定日程
截止日期：2025-03-07
重要程度：5

事项: 洗漱
日期: 2025-03-08
时间段: 06:00-06:30
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 早餐
日期: 2025-03-08
时间段: 07:00-07:30
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 午餐
日期: 2025-03-08
时间段: 12:00-13:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 晚餐
日期: 2025-03-08
时间段: 18:00-19:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 睡觉
日期: 2025-03-08
时间段: 22:00-06:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 洗漱
日期: 2025-03-09
时间段: 06:00-06:30
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 早餐
日期: 2025-03-09
时间段: 07:00-07:30
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 午餐
日期: 2025-03-09
时间段: 12:00-13:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 晚餐
日期: 2025-03-09
时间段: 18:00-19:00
类型: 任务事项
截止日期：2025-03-08
重要程度：5

事项: 睡觉
日期: 2025-03-09
时间段: 22:00-06:00
类型: 任务事项
截止日期

In [5]:
# 使用当前时间删除过期事件
result = processor.delete_past_events()

# 查看删除结果
print(f"删除了 {result['deleted_count']} 个事件")
for event in result['deleted_events']:
    print(f"- {event['title']} ({event['date']} {event['time_range']})")

删除了 9 个事件
- 准备会议材料 (2025-03-04 08:00-10:00)
- 选择花束 (2025-03-05 18:00-18:30)
- 下单购买 (2025-03-05 18:30-19:00)
- 分析需求 (2025-03-04 10:30-11:30)
- 编写代码 (2025-03-04 11:00-13:00)
- 测试与调试 (2025-03-04 13:15-14:15)
- 吃早饭 (2025-03-04 07:00-07:30)
- 遛狗 (2025-03-04 07:30-08:00)
- 运动 (2025-03-04 20:30-21:30)
